### change path below to your gdrive path where folder is 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/competition/DSPS'

/content/drive/MyDrive/competition/DSPS


In [ ]:
## Uncomment the lines below to download training data
!gdown 'https://drive.google.com/uc?id=1I1onAA61iui4LjOxejkxioaBjU1Ic_6X'
!unzip 'training_data.zip' -d './cvat'

In [ ]:
# cvat folder should contain 3 folders: td1, td2, td3
!ls cvat 

## Uncomment and run codes below if this is the first time running this notebook

In [ ]:
# !pip install -U PyYAML

     |████████████████████████████████| 596 kB 28.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# !pip install -qr dspYolov5/requirements.txt  # install dependencies for yolov5

## Uncomment block of codes in cell below to convert cvat annotations to yolo annotations

In [ ]:
import os
from dsps.simple_fxn import cvat2yolo
# location of cvat annotations
annotation_folder = 'cvat/'
output_path = 'data/training'
val_path = 'data/validation'
for annts in os.listdir(annotation_folder):
  json_file = os.path.join(annotation_folder,annts,'annotations','instances_default.json')
  img_path = os.path.join(annotation_folder,annts,'images')
  if(os.path.isfile(json_file)):
    cvat2yolo(annts,json_file,img_path, output_path, val_path)

## Train a new model

### If not running on GPU, change runtime to GPU
________

In [ ]:
import json
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla T4)


### **Change class labels and number of classes**: 
* Edit dspYolov5/data/dsps.yaml file 
### **Change model configuration**: 
* Edit dspYolov5/models/dsps_model.yaml

### run the code below. You may get an error the first time. 

---

restart the runtime and run again. 

stop cell after total epoch have elapsed. 

In [ ]:
!python -i dspYolov5/train_dsps.py --data dspYolov5/data/dsps.yaml --cfg dspYolov5/models/dsps_model.yaml --batch-size 4 --device 0 --freeze 4 --epoch 2 --weights /content/drive/MyDrive/competition/DSPS/weights/train/exp2/weights/last.pt

### Evaluate and Visualize Model Performance

#### change weights path in the following cell: replace exp12 in the weights path with your corresponding folder-- 'weights/train/exp12/weights/best.pt'

In [ ]:
## Uncomment the lines below to download training data
!gdown 'https://drive.google.com/uc?id=1TtDGMgTyrH6bclBf76ggZk28N9aalOzc'
!unzip 'dsps_test.zip' -d './data/test_images'

In [ ]:
## create results folder if it doen't exist

In [ ]:
!python 'dspYolov5/detect_dsps.py' --weights 'weights/train/exp2/weights/last.pt' --img 640 --conf 0.25 --source data/test_images/images

### Upload submission.json file in the "results" folder to leaderboard

In [ ]:
## sample submission file content
# [{"1 (134).JPG": [[3, 365, 585, 895, 697], [2, 41, 588, 917, 2000]]}, {"1 (154).JPG": [[1, 8, 1622, 429, 2007], [6, 24, 0, 157, 867]]}]
# [3, 365, 585, 895, 697] --> ['class','x1','y1','x2','y2']
# x1,y1 - top left corner of bounding box
# x2,y2 _ bottom right corner of bounding box

### Auto Annotate and re-upload  to cvat 

In [ ]:
!python 'dspYolov5/detect_dsps.py' --weights 'weights/train/exp2/weights/best.pt' --img 640 --conf 0.25 --source data/auto_annotate/ --nosave

In [ ]:
## this will create a cvat compatible annotation file 
img_path = 'data/test_images'
csv_path = 'results/submission.csv'
output_path = 'results/instances_default.json'
csv2cvat(csv_path, img_path,output_path)